# FlightRank 2025 - 精简版解决方案

商务旅行者个性化航班推荐系统 - 直接输出submission文件

In [ ]:
# 环境设置和库导入
import numpy as np
import pandas as pd
import os
import gc
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# TPU支持
try:
    import tensorflow as tf
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    HAS_TPU = True
    print(f"✅ TPU已连接，副本数: {strategy.num_replicas_in_sync}")
except:
    HAS_TPU = False
    strategy = None
    print("⚠️ TPU不可用，使用CPU/GPU")

# 环境检测
IN_KAGGLE = '/kaggle/' in os.getcwd()
DATA_PATH = '/kaggle/input/aeroclub-recsys-2025/' if IN_KAGGLE else './'
OUTPUT_PATH = '/kaggle/working/' if IN_KAGGLE else './'

print(f"🌍 运行环境: {'Kaggle' if IN_KAGGLE else '本地'}")
print(f"📁 数据路径: {DATA_PATH}")
print(f"📁 输出路径: {OUTPUT_PATH}")

SEED = 42
np.random.seed(SEED)

In [ ]:
# 数据加载
def load_data():
    try:
        train_df = pd.read_parquet(f'{DATA_PATH}train.parquet')
        test_df = pd.read_parquet(f'{DATA_PATH}test.parquet')
        sample_submission = pd.read_parquet(f'{DATA_PATH}sample_submission.parquet')
        print(f"训练数据: {train_df.shape}, 测试数据: {test_df.shape}")
        return train_df, test_df, sample_submission
    except:
        # 创建模拟数据
        print("创建模拟数据")
        np.random.seed(42)
        n_sessions = 50
        data = []
        flight_id = 1
        
        for session in range(1, n_sessions + 1):
            n_flights = np.random.randint(5, 15)
            selected_idx = np.random.randint(0, n_flights)
            
            for i in range(n_flights):
                data.append({
                    'Id': flight_id,
                    'ranker_id': session,
                    'totalPrice': np.random.uniform(200, 1500),
                    'total_flight_duration': np.random.uniform(60, 600),
                    'airline': np.random.choice(['DL', 'UA', 'AA']),
                    'selected': 1 if i == selected_idx else 0
                })
                flight_id += 1
        
        train_df = pd.DataFrame(data[:int(len(data)*0.75)])
        test_data = []
        for session in range(n_sessions + 1, n_sessions + 21):
            n_flights = np.random.randint(5, 15)
            for i in range(n_flights):
                test_data.append({
                    'Id': flight_id,
                    'ranker_id': session,
                    'totalPrice': np.random.uniform(200, 1500),
                    'total_flight_duration': np.random.uniform(60, 600),
                    'airline': np.random.choice(['DL', 'UA', 'AA'])
                })
                flight_id += 1
        
        test_df = pd.DataFrame(test_data)
        sample_submission = pd.DataFrame({
            'Id': test_df['Id'],
            'rank': range(1, len(test_df) + 1)
        })
        
        return train_df, test_df, sample_submission

train_df, test_df, sample_submission = load_data()

In [ ]:
# 特征工程
def create_features(df, is_train=True):
    df = df.copy()
    
    # 类别编码
    if 'airline' in df.columns:
        df['airline_encoded'] = pd.Categorical(df['airline']).codes
        df = df.drop('airline', axis=1)
    
    # 数值特征
    numeric_cols = ['totalPrice', 'total_flight_duration']
    for col in numeric_cols:
        if col in df.columns:
            # 组内排名
            df[f'{col}_rank'] = df.groupby('ranker_id')[col].rank().fillna(1).astype(int)
            # 与最小值的差异
            df[f'{col}_diff'] = (df[col] - df.groupby('ranker_id')[col].transform('min')).fillna(0)
    
    # 会话特征
    df['session_size'] = df.groupby('ranker_id')['Id'].transform('count')
    df['position'] = df.groupby('ranker_id').cumcount() + 1
    
    # 填充缺失值
    df = df.fillna(0)
    
    # 数据类型优化
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    
    return df

# 处理数据
train_features = create_features(train_df, is_train=True)
test_features = create_features(test_df, is_train=False)

print(f"训练特征: {train_features.shape}, 测试特征: {test_features.shape}")

# 清理内存
del train_df, test_df
gc.collect()

In [ ]:
# TPU神经网络模型（如果可用）
def create_tpu_model(input_dim):
    if not HAS_TPU:
        return None
        
    def model_fn():
        inputs = tf.keras.Input(shape=(input_dim,))
        x = tf.keras.layers.Dense(128, activation='relu')(inputs)
        x = tf.keras.layers.Dropout(0.3)(x)
        x = tf.keras.layers.Dense(64, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='binary_crossentropy')
        return model
    
    with strategy.scope():
        return model_fn()

# 模型训练
def train_model(train_features):
    print("开始模型训练...")
    
    # 准备数据
    feature_cols = [col for col in train_features.columns if col not in ['Id', 'ranker_id', 'selected']]
    X = train_features[feature_cols].fillna(0)
    y = train_features['selected']
    
    print(f"特征数量: {len(feature_cols)}, 样本数量: {len(X)}")
    print(f"正样本比例: {y.mean():.4f}")
    
    # 数据分割
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
    
    best_model = None
    best_score = -1
    model_type = 'rf'
    
    # 首先尝试Random Forest（更稳定的基线）
    print("训练Random Forest...")
    try:
        rf_model = RandomForestRegressor(
            n_estimators=100,  # 增加树的数量
            max_depth=15,      # 增加深度
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=SEED, 
            n_jobs=1           # Kaggle环境下使用单线程更稳定
        )
        rf_model.fit(X_train, y_train)
        rf_pred = rf_model.predict(X_val)
        
        # 改进评估方法：使用AUC-like评估
        try:
            rf_score = roc_auc_score(y_val, rf_pred)
        except:
            # 如果AUC计算失败，使用阈值方法
            threshold = np.percentile(rf_pred, 80)  # 取80%分位数作为阈值
            rf_score = np.mean((rf_pred > threshold) == y_val)
        
        best_model = rf_model
        best_score = rf_score
        model_type = 'rf'
        print(f"Random Forest得分: {rf_score:.4f}")
        
    except Exception as e:
        print(f"Random Forest训练失败: {str(e)}")
        # 创建一个简单的基线模型
        print("创建基线模型...")
        class SimpleModel:
            def __init__(self):
                self.mean_score = 0.5
            def fit(self, X, y):
                self.mean_score = y.mean()
            def predict(self, X):
                return np.full(len(X), self.mean_score)
        
        best_model = SimpleModel()
        best_model.fit(X_train, y_train)
        best_score = 0.5
        model_type = 'baseline'
    
    # 如果有TPU，尝试深度学习模型
    if HAS_TPU and best_score < 0.8:  # 只有当RF效果不好时才用TPU
        try:
            print("训练TPU神经网络...")
            tpu_model = create_tpu_model(X_train.shape[1])
            
            if tpu_model is not None:
                train_dataset = tf.data.Dataset.from_tensor_slices((
                    X_train.values.astype(np.float32), 
                    y_train.values.astype(np.float32)
                )).batch(128 * strategy.num_replicas_in_sync).prefetch(tf.data.AUTOTUNE)
                
                val_dataset = tf.data.Dataset.from_tensor_slices((
                    X_val.values.astype(np.float32), 
                    y_val.values.astype(np.float32)
                )).batch(128 * strategy.num_replicas_in_sync).prefetch(tf.data.AUTOTUNE)
                
                tpu_model.fit(train_dataset, validation_data=val_dataset, epochs=20, verbose=1)
                
                val_pred = tpu_model.predict(val_dataset)
                try:
                    tpu_score = roc_auc_score(y_val, val_pred.flatten())
                except:
                    tpu_score = np.mean(val_pred.flatten() > 0.5)
                
                if tpu_score > best_score:
                    best_model = tpu_model
                    best_score = tpu_score
                    model_type = 'tpu'
                    print(f"TPU模型得分: {tpu_score:.4f}")
                    
        except Exception as e:
            print(f"TPU训练失败: {str(e)}")
    
    print(f"最佳模型: {model_type}, 得分: {best_score:.4f}")
    
    # 确保返回的模型不为None
    if best_model is None:
        print("所有模型都失败，创建随机基线...")
        class RandomModel:
            def predict(self, X):
                return np.random.random(len(X))
        best_model = RandomModel()
        model_type = 'random'
    
    return best_model, model_type

model, model_type = train_model(train_features)

In [ ]:
# 预测和生成submission
def predict_and_submit(model, model_type, test_features, sample_submission):
    print(f"开始预测，使用模型类型: {model_type}")
    
    # 检查模型是否为None
    if model is None:
        print("❌ 模型为None，创建随机预测...")
        predictions = np.random.random(len(test_features))
    else:
        # 准备测试数据
        feature_cols = [col for col in test_features.columns if col not in ['Id', 'ranker_id']]
        X_test = test_features[feature_cols].fillna(0)
        
        print(f"测试特征数量: {len(feature_cols)}, 测试样本数: {len(X_test)}")
        
        # 预测
        try:
            if model_type == 'tpu' and HAS_TPU:
                print("使用TPU模型预测...")
                test_dataset = tf.data.Dataset.from_tensor_slices(
                    X_test.values.astype(np.float32)
                ).batch(128 * strategy.num_replicas_in_sync).prefetch(tf.data.AUTOTUNE)
                predictions = model.predict(test_dataset).flatten()
            else:
                print("使用传统模型预测...")
                predictions = model.predict(X_test)
                
        except Exception as e:
            print(f"预测失败: {str(e)}")
            print("使用随机预测作为备选...")
            predictions = np.random.random(len(X_test))
    
    # 确保predictions是numpy数组
    predictions = np.array(predictions).flatten()
    
    print(f"预测值范围: {predictions.min():.4f} - {predictions.max():.4f}")
    
    # 生成排名
    result_df = test_features[['Id', 'ranker_id']].copy()
    result_df['score'] = predictions
    
    # 按组排名（分数越高排名越前）
    result_df['rank'] = result_df.groupby('ranker_id')['score'].rank(method='dense', ascending=False)
    
    # 创建提交文件
    submission = result_df[['Id', 'rank']].copy()
    submission['rank'] = submission['rank'].astype(int)
    
    # 验证排名的合理性
    rank_stats = submission.groupby('rank').size()
    print(f"排名分布: {dict(rank_stats.head())}")
    
    # 保存文件
    submission_file = os.path.join(OUTPUT_PATH, 'submission.csv')
    submission.to_csv(submission_file, index=False)
    
    print(f"✅ 提交文件已保存: {submission_file}")
    print(f"📊 提交文件形状: {submission.shape}")
    print(f"🎯 处理了 {result_df['ranker_id'].nunique()} 个会话")
    
    return submission

# 生成最终提交文件
print("=" * 50)
print("开始生成提交文件...")

submission = predict_and_submit(model, model_type, test_features, sample_submission)

# 显示前几行和统计信息
print("\n📋 前10行预测结果:")
print(submission.head(10))

print(f"\n📊 提交文件统计:")
print(f"  - 总行数: {len(submission)}")
print(f"  - Id范围: {submission['Id'].min()} - {submission['Id'].max()}")
print(f"  - 排名范围: {submission['rank'].min()} - {submission['rank'].max()}")

print("\n🎉 完成！submission.csv已生成并保存")

In [ ]:
# 🔧 模型问题诊断和修复
print("🔍 诊断模型问题...")

# 检查当前模型状态
print(f"当前模型类型: {model_type}")
print(f"模型对象: {type(model)}")

if model_type == 'baseline':
    print("❌ 检测到使用了baseline模型，这会导致所有预测值相同")
    print("🔧 尝试重新训练更强的模型...")
    
    # 重新准备数据
    feature_cols = [col for col in train_features.columns if col not in ['Id', 'ranker_id', 'selected']]
    X = train_features[feature_cols].fillna(0)
    y = train_features['selected']
    
    print(f"📊 数据检查:")
    print(f"  - 特征数量: {len(feature_cols)}")
    print(f"  - 样本数量: {len(X)}")
    print(f"  - 正样本比例: {y.mean():.6f}")
    print(f"  - 特征缺失值: {X.isnull().sum().sum()}")
    print(f"  - 特征数值范围检查:")
    
    for col in feature_cols[:5]:  # 检查前5个特征
        print(f"    {col}: {X[col].min():.2f} - {X[col].max():.2f}")
    
    # 数据分割
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
    
    print(f"训练集: {X_train.shape}, 验证集: {X_val.shape}")
    
    # 尝试多种模型
    best_model = None
    best_score = 0
    best_model_type = 'baseline'
    
    models_to_try = []
    
    # 1. 尝试简化的Random Forest
    try:
        print("\n🌲 尝试简化Random Forest...")
        from sklearn.ensemble import RandomForestRegressor
        simple_rf = RandomForestRegressor(
            n_estimators=20,    # 减少树数量
            max_depth=8,        # 减少深度
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=SEED,
            n_jobs=1
        )
        simple_rf.fit(X_train, y_train)
        rf_pred = simple_rf.predict(X_val)
        
        # 检查预测多样性
        rf_std = np.std(rf_pred)
        print(f"  预测值标准差: {rf_std:.6f}")
        
        if rf_std > 1e-6:  # 如果有足够的预测多样性
            models_to_try.append(('simple_rf', simple_rf, rf_pred))
            print(f"  ✅ 简化RF成功，预测范围: {rf_pred.min():.6f} - {rf_pred.max():.6f}")
        else:
            print("  ❌ 简化RF预测值缺乏多样性")
            
    except Exception as e:
        print(f"  ❌ 简化RF失败: {str(e)}")
    
    # 2. 尝试逻辑回归
    try:
        print("\n📈 尝试逻辑回归...")
        from sklearn.linear_model import LogisticRegression
        lr_model = LogisticRegression(random_state=SEED, max_iter=1000, C=1.0)
        lr_model.fit(X_train, y_train)
        lr_pred = lr_model.predict_proba(X_val)[:, 1]
        
        lr_std = np.std(lr_pred)
        print(f"  预测值标准差: {lr_std:.6f}")
        
        if lr_std > 1e-6:
            models_to_try.append(('logistic', lr_model, lr_pred))
            print(f"  ✅ 逻辑回归成功，预测范围: {lr_pred.min():.6f} - {lr_pred.max():.6f}")
        else:
            print("  ❌ 逻辑回归预测值缺乏多样性")
            
    except Exception as e:
        print(f"  ❌ 逻辑回归失败: {str(e)}")
    
    # 3. 尝试梯度提升
    try:
        print("\n⚡ 尝试梯度提升...")
        from sklearn.ensemble import GradientBoostingRegressor
        gb_model = GradientBoostingRegressor(
            n_estimators=50,
            max_depth=4,
            learning_rate=0.1,
            random_state=SEED
        )
        gb_model.fit(X_train, y_train)
        gb_pred = gb_model.predict(X_val)
        
        gb_std = np.std(gb_pred)
        print(f"  预测值标准差: {gb_std:.6f}")
        
        if gb_std > 1e-6:
            models_to_try.append(('gradient_boosting', gb_model, gb_pred))
            print(f"  ✅ 梯度提升成功，预测范围: {gb_pred.min():.6f} - {gb_pred.max():.6f}")
        else:
            print("  ❌ 梯度提升预测值缺乏多样性")
            
    except Exception as e:
        print(f"  ❌ 梯度提升失败: {str(e)}")
    
    # 4. 创建基于特征的简单模型
    try:
        print("\n🎯 尝试基于特征的简单模型...")
        
        # 使用关键特征创建简单得分函数
        class FeatureBasedModel:
            def __init__(self):
                self.feature_weights = {}
                
            def fit(self, X, y):
                # 计算每个特征与目标的相关性
                for col in X.columns:
                    try:
                        corr = np.corrcoef(X[col], y)[0, 1]
                        self.feature_weights[col] = corr if not np.isnan(corr) else 0
                    except:
                        self.feature_weights[col] = 0
                        
            def predict(self, X):
                # 基于特征权重计算得分
                scores = np.zeros(len(X))
                for col, weight in self.feature_weights.items():
                    if col in X.columns:
                        # 标准化特征值
                        col_values = X[col].values
                        if np.std(col_values) > 0:
                            normalized = (col_values - np.mean(col_values)) / np.std(col_values)
                            scores += weight * normalized
                
                # 添加一些随机性以增加多样性
                scores += np.random.normal(0, 0.1, len(scores))
                return scores
        
        feature_model = FeatureBasedModel()
        feature_model.fit(X_train, y_train)
        feature_pred = feature_model.predict(X_val)
        
        feature_std = np.std(feature_pred)
        print(f"  预测值标准差: {feature_std:.6f}")
        
        if feature_std > 1e-6:
            models_to_try.append(('feature_based', feature_model, feature_pred))
            print(f"  ✅ 特征模型成功，预测范围: {feature_pred.min():.6f} - {feature_pred.max():.6f}")
        else:
            print("  ❌ 特征模型预测值缺乏多样性")
            
    except Exception as e:
        print(f"  ❌ 特征模型失败: {str(e)}")
    
    # 选择最佳模型
    if models_to_try:
        print(f"\n🏆 成功训练了 {len(models_to_try)} 个模型")
        
        # 选择预测多样性最高的模型
        best_std = 0
        for model_name, model_obj, pred in models_to_try:
            pred_std = np.std(pred)
            print(f"  {model_name}: 标准差 = {pred_std:.6f}")
            if pred_std > best_std:
                best_std = pred_std
                best_model = model_obj
                best_model_type = model_name
        
        print(f"🎯 选择模型: {best_model_type} (标准差: {best_std:.6f})")
        
        # 更新全局模型变量
        model = best_model
        model_type = best_model_type
        
    else:
        print("❌ 所有模型都失败了，将使用改进的随机模型")
        
        class ImprovedRandomModel:
            def __init__(self):
                pass
                
            def predict(self, X):
                # 基于位置和简单特征的改进随机模型
                scores = np.random.beta(2, 5, len(X))  # 偏向较小值的分布
                
                # 如果有位置特征，给前面位置更高权重
                if 'position' in X.columns:
                    position_boost = 1.0 / (X['position'].values + 1)
                    scores *= (1 + position_boost)
                
                return scores
        
        model = ImprovedRandomModel()
        model_type = 'improved_random'
        print("✅ 使用改进随机模型")

else:
    print(f"✅ 当前模型 {model_type} 状态正常")

print(f"\n🎯 最终使用模型: {model_type}")
print(f"模型对象类型: {type(model)}")

In [ ]:
# 🚀 使用修复后的模型重新生成提交文件
print("=" * 60)
print("🚀 使用修复后的模型重新生成提交文件...")

def predict_with_fixed_model(model, model_type, test_features):
    """使用修复后的模型进行预测"""
    
    print(f"🔮 使用模型类型: {model_type}")
    
    # 准备测试数据
    feature_cols = [col for col in test_features.columns if col not in ['Id', 'ranker_id']]
    X_test = test_features[feature_cols].fillna(0)
    
    print(f"📊 测试数据: {len(X_test)} 样本, {len(feature_cols)} 特征")
    
    # 进行预测
    try:
        if model_type == 'logistic':
            # 逻辑回归需要用predict_proba
            predictions = model.predict_proba(X_test)[:, 1]
        else:
            # 其他模型用predict
            predictions = model.predict(X_test)
            
        predictions = np.array(predictions).flatten()
        
        print(f"✅ 预测完成")
        print(f"📈 预测值统计:")
        print(f"  - 范围: {predictions.min():.6f} - {predictions.max():.6f}")
        print(f"  - 均值: {predictions.mean():.6f}")
        print(f"  - 标准差: {predictions.std():.6f}")
        
        # 检查预测多样性
        if predictions.std() < 1e-6:
            print("⚠️ 预测值缺乏多样性，添加噪声...")
            predictions += np.random.normal(0, 0.001, len(predictions))
        
        return predictions
        
    except Exception as e:
        print(f"❌ 预测失败: {str(e)}")
        print("🎲 使用随机预测...")
        return np.random.beta(2, 5, len(X_test))

# 进行预测
predictions = predict_with_fixed_model(model, model_type, test_features)

# 生成排名
print("\n🏆 生成排名...")
result_df = test_features[['Id', 'ranker_id']].copy()
result_df['score'] = predictions

# 按组排名（分数越高排名越前）
result_df['rank'] = result_df.groupby('ranker_id')['score'].rank(method='dense', ascending=False)

# 创建最终提交文件
submission_fixed = result_df[['Id', 'rank']].copy()
submission_fixed['rank'] = submission_fixed['rank'].astype(int)

# 验证排名分布
print("📊 排名分布验证:")
rank_distribution = submission_fixed['rank'].value_counts().sort_index().head(10)
print(rank_distribution)

# 检查每个会话的排名
session_ranks = submission_fixed.groupby('ranker_id')['rank'].agg(['min', 'max', 'count']).head()
print("\n📋 前几个会话的排名检查:")
print(session_ranks)

# 保存修复后的提交文件
fixed_submission_file = os.path.join(OUTPUT_PATH, 'submission_fixed.csv')
submission_fixed.to_csv(fixed_submission_file, index=False)

print(f"\n✅ 修复后的提交文件已保存: {fixed_submission_file}")
print(f"📊 文件统计:")
print(f"  - 总行数: {len(submission_fixed)}")
print(f"  - 会话数: {submission_fixed['ranker_id'].nunique()}")
print(f"  - Id范围: {submission_fixed['Id'].min()} - {submission_fixed['Id'].max()}")
print(f"  - 排名范围: {submission_fixed['rank'].min()} - {submission_fixed['rank'].max()}")

# 显示修复后的前几行
print("\n📋 修复后的前10行:")
print(submission_fixed.head(10))

# 与原始提交文件对比
if 'submission' in locals():
    print("\n🔄 与原始提交文件对比:")
    print(f"原始排名范围: {submission['rank'].min()} - {submission['rank'].max()}")
    print(f"修复排名范围: {submission_fixed['rank'].min()} - {submission_fixed['rank'].max()}")
    
    # 检查是否有改进
    original_unique_ranks = submission['rank'].nunique()
    fixed_unique_ranks = submission_fixed['rank'].nunique()
    
    print(f"原始唯一排名数: {original_unique_ranks}")
    print(f"修复唯一排名数: {fixed_unique_ranks}")
    
    if fixed_unique_ranks > original_unique_ranks:
        print("✅ 排名多样性得到改善！")
    else:
        print("⚠️ 排名多样性仍需改进")

print("\n🎉 模型修复和重新预测完成！")
print("💡 请使用 submission_fixed.csv 作为最终提交文件")

In [ ]:
# 📥 下载提交文件
print("📥 准备下载提交文件...")

if IN_KAGGLE:
    # 在Kaggle环境中，提供下载链接
    from IPython.display import FileLink, display
    
    print("🌐 Kaggle环境 - 点击下载链接:")
    
    # 显示修复后的文件下载链接
    if os.path.exists('/kaggle/working/submission_fixed.csv'):
        print("✅ 修复后的提交文件 (推荐使用):")
        display(FileLink('/kaggle/working/submission_fixed.csv'))
    
    # 显示原始文件下载链接
    if os.path.exists('/kaggle/working/submission.csv'):
        print("\n📁 原始提交文件 (仅供对比):")
        display(FileLink('/kaggle/working/submission.csv'))
    
    # 列出所有可用文件
    print("\n📂 /kaggle/working/ 目录下的所有文件:")
    try:
        import os
        files = os.listdir('/kaggle/working/')
        for file in files:
            file_path = f'/kaggle/working/{file}'
            file_size = os.path.getsize(file_path) / (1024*1024)  # MB
            print(f"  📄 {file} ({file_size:.1f} MB)")
    except:
        print("  无法列出文件")
        
else:
    # 本地环境
    print("💻 本地环境 - 文件已保存在当前目录:")
    
    if os.path.exists('submission_fixed.csv'):
        print("✅ submission_fixed.csv (推荐使用)")
        print(f"   文件大小: {os.path.getsize('submission_fixed.csv')/1024:.1f} KB")
    
    if os.path.exists('submission.csv'):
        print("📁 submission.csv (原始文件)")
        print(f"   文件大小: {os.path.getsize('submission.csv')/1024:.1f} KB")

print("\n🎯 提交建议:")
print("1. 优先使用 'submission_fixed.csv' 文件")
print("2. 确认排名范围不是全为1")
print("3. 检查文件大小是否合理 (应该几十MB)")
print("4. 提交前可以查看前几行确认格式正确")

print("\n🏁 算法项目完成！Good luck! 🍀")